# Demand Forecasting

Demand forecasting is a type of machine learning task in which we try to predict the future value of a certain domain. In this experiment, we are trying to forecast the demand value by looking at sales data provided for our retail service.

# Problem Statement

Weekly demand forecasting for every menu group.

# Libraries

In [1]:
import pandas as pd
import numpy as np

import holidays
import xgboost as xgb
from sklearn.metrics import r2_score
from sklearn.preprocessing import OneHotEncoder
from sklearn.model_selection import train_test_split, GridSearchCV

# Initial EDA

Here, will be done some brief analysis regarding the overall data that is provided.

In [2]:
raw = pd.read_csv("data.csv")

In [3]:
raw.head(n=10)

,salesdate,menuid,menuname,menu_group,qty_total
0,2022-10-09,518,Premium Beef Deal,0,1
1,2022-10-09,518,Premium Beef Deal,0,1
2,2022-10-09,518,Premium Beef Deal,0,1
3,2022-10-09,519,[FS] Spaghetti Prawn/Chicken Aglio Olio,Spaghetti Prawn/Chicken Aglio Olio,1
4,2022-10-09,519,[FS] Spaghetti Prawn/Chicken Aglio Olio,Spaghetti Prawn/Chicken Aglio Olio,1
5,2022-10-09,519,[FS] Spaghetti Prawn/Chicken Aglio Olio,Spaghetti Prawn/Chicken Aglio Olio,1
6,2022-10-09,519,[FS] Spaghetti Prawn/Chicken Aglio Olio,Spaghetti Prawn/Chicken Aglio Olio,1
7,2022-10-09,519,[FS] Spaghetti Prawn/Chicken Aglio Olio,Spaghetti Prawn/Chicken Aglio Olio,1
8,2022-10-09,519,[FS] Spaghetti Prawn/Chicken Aglio Olio,Spaghetti Prawn/Chicken Aglio Olio,1
9,2022-10-09,519,[FS] Spaghetti Prawn/Chicken Aglio Olio,Spaghetti Prawn/Chicken Aglio Olio,1


In [4]:
raw.tail(n=10)

,salesdate,menuid,menuname,menu_group,qty_total
8892,2022-12-02,577,Sweet and Spicy Chicken Rice,Sweet and Spicy Chicken,1
8893,2022-12-02,578,(ALC)Sweet and Spicy Chicken,Sweet and Spicy Chicken,1
8894,2022-12-02,84,Extra Smoked Beef,Extra Item,1
8895,2022-12-02,84,Extra Smoked Beef,Extra Item,1
8896,2022-12-02,436,Plastic Spoon,0,2
8897,2022-12-02,438,Plastic Spoon,0,2
8898,2022-12-02,236,Black Pepper Chicken Rice,Black Pepper Chicken,1
8899,2022-12-02,242,(ALC)Classic Mentega Chicken,Classic Mentega Chicken,1
8900,2022-12-02,244,(ALC)Kung Pao Chicken,Kung Pao Chicken,1
8901,2022-12-02,245,(ALC)Buttermilk Chicken,Buttermilk Chicken,1


The data is provided in an expanded format of time-series data, which is a daily sales broken down by its menu identifiers.

From problem statement, we know that same menu name doesn't equal to same menu id, and vice versa.

In the data we can also see that the menu group of each data can be 0 and can be part of a named menu. Menu group of 0 probably means that the data does not really belong to any menu group and can probably seen as an a la carte menu. There are also items that have `Extra Item` as the menu group, meaning that this item is an addition of another menu.

From the menu name we can see that there are several types of menu. Specifically, we will be using the [FS] for menu that is in `Flash Sales` and `B1G1` for menu that is in `Buy 1 Get 1` Promo. There are several others that can actually be used as features, such as `Tambah xRB dapat y` (adding other items valued as x will get a bonus item of y), `x Weekly Best Seller` (item x is a weekly best seller`, and etc.. However, only the first two will be used for the sake of demonstration.

# Preprocess

## Changing the type of dates into datetime for easier processing

In [5]:
raw.shape

(8902, 5)

In [6]:
raw.dtypes

salesdate     object
menuid         int64
menuname      object
menu_group    object
qty_total      int64
dtype: object

In [7]:
raw['salesdate'] = pd.to_datetime(raw['salesdate'], format='%Y-%m-%d')
raw.dtypes

salesdate     datetime64[ns]
menuid                 int64
menuname              object
menu_group            object
qty_total              int64
dtype: object

## Missing Values

In [8]:
# No missing value detected
raw.isnull().sum()

salesdate     0
menuid        0
menuname      0
menu_group    0
qty_total     0
dtype: int64

## Checking Range of Data

In [9]:
raw.salesdate.min()

Timestamp('2022-05-16 00:00:00')

In [10]:
raw.salesdate.max()

Timestamp('2023-05-15 00:00:00')

We see that the data consists of data from May 2022 to May 2023. However, the data is not fully day-to-day data, meaning that there are some dates that is skipped. For example, the data can go directly from 2023-05-07 to 2023-05-15.

## Aggregating Data

We will want to aggregate the data based on its menu group on date level. The quantity will be summed, and each data in this context will be the total sales of the menu group on the specified sales date.

In [11]:
data = raw.pivot_table('qty_total', ['salesdate', 'menu_group'], aggfunc='sum').reset_index()

In [12]:
data

,salesdate,menu_group,qty_total
0,2022-05-16,0,24
1,2022-05-16,Extra Item,6
2,2022-05-16,Mentai Gyudon,1
3,2022-05-16,Miso Gyudon,1
4,2022-05-16,Sei Ayam Longboard,1
...,...,...,...
1271,2023-05-15,Extra Item,2
1272,2023-05-15,Sei Ayam Juicy,2
1273,2023-05-15,Spaghetti Chicken Malaysian Kari,1
1274,2023-05-15,Spaghetti Chicken Singapore Laksa,3


### Adding list of menus in the menu group

We will also add the menu name of the menu group for reference later on.

In [13]:
def generate_menus(data, raw_data):
    filtered = raw_data[(raw_data['salesdate'] == data['salesdate']) & (raw_data['menu_group'] == data['menu_group'])]
    menu_names = filtered['menuname'].to_list()
    return menu_names

In [14]:
data['menus'] = data.apply(lambda x: generate_menus(x, raw), axis=1)

In [15]:
data

,salesdate,menu_group,qty_total,menus
0,2022-05-16,0,24,"[Sayur Daun Singkong + Telur Barendo - Small, ..."
1,2022-05-16,Extra Item,6,"[Extra Sayur Singkong, Extra Nasi Putih, Extra..."
2,2022-05-16,Mentai Gyudon,1,[Mentai Gyudon Aburi]
3,2022-05-16,Miso Gyudon,1,[Miso Gyudon Aburi]
4,2022-05-16,Sei Ayam Longboard,1,[Sei Ayam Longboard]
...,...,...,...,...
1271,2023-05-15,Extra Item,2,[Extra Sei Sapi Premium (30gr)]
1272,2023-05-15,Sei Ayam Juicy,2,[Extra Sei Ayam Juicy (30gr)]
1273,2023-05-15,Spaghetti Chicken Malaysian Kari,1,[Spaghetti Chicken Malaysian Kari]
1274,2023-05-15,Spaghetti Chicken Singapore Laksa,3,"[Spaghetti Chicken Singapore Laksa, Spaghetti ..."


# Feature Engineering

## Adding Time Features

In [16]:
data['year'] = data['salesdate'].dt.year
data['month'] = data['salesdate'].dt.month
data['week'] = data['salesdate'].dt.isocalendar().week.astype("int64")
data['day_of_month'] = data['salesdate'].dt.day
data['day_of_week'] = data['salesdate'].dt.dayofweek
data['day_of_year'] = data['salesdate'].dt.dayofyear

In [17]:
data

,salesdate,menu_group,qty_total,menus,year,month,week,day_of_month,day_of_week,day_of_year
0,2022-05-16,0,24,"[Sayur Daun Singkong + Telur Barendo - Small, ...",2022,5,20,16,0,136
1,2022-05-16,Extra Item,6,"[Extra Sayur Singkong, Extra Nasi Putih, Extra...",2022,5,20,16,0,136
2,2022-05-16,Mentai Gyudon,1,[Mentai Gyudon Aburi],2022,5,20,16,0,136
3,2022-05-16,Miso Gyudon,1,[Miso Gyudon Aburi],2022,5,20,16,0,136
4,2022-05-16,Sei Ayam Longboard,1,[Sei Ayam Longboard],2022,5,20,16,0,136
...,...,...,...,...,...,...,...,...,...,...
1271,2023-05-15,Extra Item,2,[Extra Sei Sapi Premium (30gr)],2023,5,20,15,0,135
1272,2023-05-15,Sei Ayam Juicy,2,[Extra Sei Ayam Juicy (30gr)],2023,5,20,15,0,135
1273,2023-05-15,Spaghetti Chicken Malaysian Kari,1,[Spaghetti Chicken Malaysian Kari],2023,5,20,15,0,135
1274,2023-05-15,Spaghetti Chicken Singapore Laksa,3,"[Spaghetti Chicken Singapore Laksa, Spaghetti ...",2023,5,20,15,0,135


## Adding Holiday and Weekend Check

### Holiday Check

In [18]:
data['is_holiday'] = data.apply(lambda x: x['salesdate'] in holidays.country_holidays('ID'), axis=1)

In [19]:
data

,salesdate,menu_group,qty_total,menus,year,month,week,day_of_month,day_of_week,day_of_year,is_holiday
0,2022-05-16,0,24,"[Sayur Daun Singkong + Telur Barendo - Small, ...",2022,5,20,16,0,136,True
1,2022-05-16,Extra Item,6,"[Extra Sayur Singkong, Extra Nasi Putih, Extra...",2022,5,20,16,0,136,True
2,2022-05-16,Mentai Gyudon,1,[Mentai Gyudon Aburi],2022,5,20,16,0,136,True
3,2022-05-16,Miso Gyudon,1,[Miso Gyudon Aburi],2022,5,20,16,0,136,True
4,2022-05-16,Sei Ayam Longboard,1,[Sei Ayam Longboard],2022,5,20,16,0,136,True
...,...,...,...,...,...,...,...,...,...,...,...
1271,2023-05-15,Extra Item,2,[Extra Sei Sapi Premium (30gr)],2023,5,20,15,0,135,False
1272,2023-05-15,Sei Ayam Juicy,2,[Extra Sei Ayam Juicy (30gr)],2023,5,20,15,0,135,False
1273,2023-05-15,Spaghetti Chicken Malaysian Kari,1,[Spaghetti Chicken Malaysian Kari],2023,5,20,15,0,135,False
1274,2023-05-15,Spaghetti Chicken Singapore Laksa,3,"[Spaghetti Chicken Singapore Laksa, Spaghetti ...",2023,5,20,15,0,135,False


### Weekend Check

In [20]:
data['is_weekend'] = data.apply(lambda x: x['day_of_week'] in [5, 6], axis=1)

In [21]:
data.head(10)

,salesdate,menu_group,qty_total,menus,year,month,week,day_of_month,day_of_week,day_of_year,is_holiday,is_weekend
0,2022-05-16,0,24,"[Sayur Daun Singkong + Telur Barendo - Small, ...",2022,5,20,16,0,136,True,False
1,2022-05-16,Extra Item,6,"[Extra Sayur Singkong, Extra Nasi Putih, Extra...",2022,5,20,16,0,136,True,False
2,2022-05-16,Mentai Gyudon,1,[Mentai Gyudon Aburi],2022,5,20,16,0,136,True,False
3,2022-05-16,Miso Gyudon,1,[Miso Gyudon Aburi],2022,5,20,16,0,136,True,False
4,2022-05-16,Sei Ayam Longboard,1,[Sei Ayam Longboard],2022,5,20,16,0,136,True,False
5,2022-05-16,Sei Ayam sambal matah,3,"[Sei Ayam Matah Small, Sei Ayam Matah Small]",2022,5,20,16,0,136,True,False
6,2022-05-16,Sei Sapi Belimbing Wuluh,3,"[Sei Sapi Belimbing Wuluh Regular, Sei Sapi Be...",2022,5,20,16,0,136,True,False
7,2022-05-16,Sei Sapi sambal andaliman,1,[Sei Sapi Andaliman Sambal diatas daging],2022,5,20,16,0,136,True,False
8,2022-05-16,Sei Sapi sambal matah,7,"[Sei Sapi Matah Regular, Sei Sapi Matah Small,...",2022,5,20,16,0,136,True,False
9,2022-05-16,Sei Sapi sambal rica,4,"[Sei Sapi Rica Small, Sei Sapi Rica Regular, S...",2022,5,20,16,0,136,True,False


## Adding Sales or B1G1 Product Check

In [22]:
def check_include_deals(menu_names):
    for menu in menu_names:
        if menu[:4] in ['B1G1', '[FS]']:
            return True
    return False

In [23]:
data['include_deals'] = data.apply(lambda x: check_include_deals(x['menus']), axis=1)

In [24]:
data.loc[1185]

salesdate                                      2023-04-08 00:00:00
menu_group                                       Chicken Katsu Don
qty_total                                                       11
menus            [B1G1 Chicken Katsu Don, B1G1 Chicken Katsu Do...
year                                                          2023
month                                                            4
week                                                            14
day_of_month                                                     8
day_of_week                                                      5
day_of_year                                                     98
is_holiday                                                   False
is_weekend                                                    True
include_deals                                                 True
Name: 1185, dtype: object

## Dropping Unused Features

Unused features defined here are features that is not semantically useful for the model, either because the features are not relevant or the features have been derived to generate another features.

In [25]:
data.drop(['salesdate', 'menus'], inplace=True, axis=1)

In [26]:
data

,menu_group,qty_total,year,month,week,day_of_month,day_of_week,day_of_year,is_holiday,is_weekend,include_deals
0,0,24,2022,5,20,16,0,136,True,False,False
1,Extra Item,6,2022,5,20,16,0,136,True,False,False
2,Mentai Gyudon,1,2022,5,20,16,0,136,True,False,False
3,Miso Gyudon,1,2022,5,20,16,0,136,True,False,False
4,Sei Ayam Longboard,1,2022,5,20,16,0,136,True,False,False
...,...,...,...,...,...,...,...,...,...,...,...
1271,Extra Item,2,2023,5,20,15,0,135,False,False,False
1272,Sei Ayam Juicy,2,2023,5,20,15,0,135,False,False,False
1273,Spaghetti Chicken Malaysian Kari,1,2023,5,20,15,0,135,False,False,False
1274,Spaghetti Chicken Singapore Laksa,3,2023,5,20,15,0,135,False,False,False


# Training


## Split

Splitting the dataset is a bottleneck here. This is because for time series we will have to keep the order of the data to preserve the trends in the time-related dimension. However, here we also have our menu group that should be available in both training and test set so that it can be learned by the model. This aspect is left unchecked, meaning that it is possible that some products in the test set haven't been learned by the model since it is not available in the training set.

The surest way to fix this problem is by ensuring that every menu group exists in every data generated every day, even if there is no sale for this product. This way we will have no worries regarding the problem. Aside from, possibly the ETL process generating the dataset, we can also generate the holes by imputing the data. However, this is not implemented in this experiment.

In turn, we will try to keep the order of the data while in turn not guaranting the availability of menu group in both dataset. From further inspection, it is also found that there exists a menu group that only has one data in the dataset.

Note: using `stratify` and `shuffle` in Scikit-Learn together to address each problem is not possible.

In [27]:
train, test = train_test_split(data, test_size=0.2, shuffle=False)

In [28]:
train

,menu_group,qty_total,year,month,week,day_of_month,day_of_week,day_of_year,is_holiday,is_weekend,include_deals
0,0,24,2022,5,20,16,0,136,True,False,False
1,Extra Item,6,2022,5,20,16,0,136,True,False,False
2,Mentai Gyudon,1,2022,5,20,16,0,136,True,False,False
3,Miso Gyudon,1,2022,5,20,16,0,136,True,False,False
4,Sei Ayam Longboard,1,2022,5,20,16,0,136,True,False,False
...,...,...,...,...,...,...,...,...,...,...,...
1015,Nasei Bungkus Sapi Padang,4,2023,2,9,27,0,58,False,False,False
1016,Sei Ayam Juicy,2,2023,2,9,27,0,58,False,False,False
1017,Spaghetti Samyang Carbonara,4,2023,2,9,27,0,58,False,False,False
1018,0,100,2023,2,9,28,1,59,False,False,False


In [29]:
test

,menu_group,qty_total,year,month,week,day_of_month,day_of_week,day_of_year,is_holiday,is_weekend,include_deals
1020,Bumbu Cabe Garam Chicken,5,2023,2,9,28,1,59,False,False,False
1021,Buttermilk Chicken,1,2023,2,9,28,1,59,False,False,False
1022,Caramelized Butter Chicken,8,2023,2,9,28,1,59,False,False,False
1023,Chicken Katsu Don,48,2023,2,9,28,1,59,False,False,True
1024,Chicken Skin,16,2023,2,9,28,1,59,False,False,False
...,...,...,...,...,...,...,...,...,...,...,...
1271,Extra Item,2,2023,5,20,15,0,135,False,False,False
1272,Sei Ayam Juicy,2,2023,5,20,15,0,135,False,False,False
1273,Spaghetti Chicken Malaysian Kari,1,2023,5,20,15,0,135,False,False,False
1274,Spaghetti Chicken Singapore Laksa,3,2023,5,20,15,0,135,False,False,False


## Encoding

For our categorical data, menu group, since it is not ordinal and doesn't have any order to its value, we should use One Hot Encoding / generating dummies with Pandas to encode the data. However, we will have many more features generated because of this.

**This approach should handle the encoding by applying PCA to reduce features and avoid curse of dimensionality. Another approach can be used, for example with different encoding methods, such as entity embedding.**

### OHE for Training Set

In [30]:
encoder = OneHotEncoder(handle_unknown='ignore')
encoded = encoder.fit_transform(train[['menu_group']])
train[encoder.categories_[0]] = encoded.toarray()

In [31]:
train

,menu_group,qty_total,year,month,week,day_of_month,day_of_week,day_of_year,is_holiday,is_weekend,...,Spaghetti Prawn/Chicken Aglio Olio,Spaghetti Samyang Carbonara,Spaghetti Smoked Beef Garlic Butter,Spaghetti Spicy Seafood Mentai,Spaghetti with Truffle Cream & Chicken,Spaghetti with Truffle Cream and Chicken,Spicy Gyudon,Sweet and Spicy Chicken,Teriyaki Gyudon,Truffle Gyudon
0,0,24,2022,5,20,16,0,136,True,False,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,Extra Item,6,2022,5,20,16,0,136,True,False,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,Mentai Gyudon,1,2022,5,20,16,0,136,True,False,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,Miso Gyudon,1,2022,5,20,16,0,136,True,False,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,Sei Ayam Longboard,1,2022,5,20,16,0,136,True,False,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1015,Nasei Bungkus Sapi Padang,4,2023,2,9,27,0,58,False,False,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1016,Sei Ayam Juicy,2,2023,2,9,27,0,58,False,False,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1017,Spaghetti Samyang Carbonara,4,2023,2,9,27,0,58,False,False,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1018,0,100,2023,2,9,28,1,59,False,False,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


### OHE for Test Set

In [32]:
encoded = encoder.transform(test[['menu_group']])
test[encoder.categories_[0]] = encoded.toarray()

In [33]:
test

,menu_group,qty_total,year,month,week,day_of_month,day_of_week,day_of_year,is_holiday,is_weekend,...,Spaghetti Prawn/Chicken Aglio Olio,Spaghetti Samyang Carbonara,Spaghetti Smoked Beef Garlic Butter,Spaghetti Spicy Seafood Mentai,Spaghetti with Truffle Cream & Chicken,Spaghetti with Truffle Cream and Chicken,Spicy Gyudon,Sweet and Spicy Chicken,Teriyaki Gyudon,Truffle Gyudon
1020,Bumbu Cabe Garam Chicken,5,2023,2,9,28,1,59,False,False,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1021,Buttermilk Chicken,1,2023,2,9,28,1,59,False,False,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1022,Caramelized Butter Chicken,8,2023,2,9,28,1,59,False,False,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1023,Chicken Katsu Don,48,2023,2,9,28,1,59,False,False,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1024,Chicken Skin,16,2023,2,9,28,1,59,False,False,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1271,Extra Item,2,2023,5,20,15,0,135,False,False,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1272,Sei Ayam Juicy,2,2023,5,20,15,0,135,False,False,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1273,Spaghetti Chicken Malaysian Kari,1,2023,5,20,15,0,135,False,False,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1274,Spaghetti Chicken Singapore Laksa,3,2023,5,20,15,0,135,False,False,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


### Dropping Encoded Columns

In [34]:
train.drop(['menu_group'], axis=1, inplace=True)
test.drop(['menu_group'], axis=1, inplace=True)

## Training

### Data

In [35]:
X_train = train.drop(['qty_total'], axis=1)
y_train = train['qty_total']

In [36]:
X_test = test.drop(['qty_total'], axis=1)
y_test = test['qty_total']

### Model

First time-series experiment with XGBoost :D Using GridSearch to find best hyperparameters.

In [65]:
params = {
    'learning_rate': [0.01, 0.001, 0.005, 0.0001],
    'max_depth': [2, 3, 4, 5]
}

In [71]:
xgb_reg = xgb.XGBRegressor(nthread=-1, objective='reg:squarederror', n_estimators=5000)
grid = GridSearchCV(xgb_reg, params)
grid.fit(X_train, y_train)

GridSearchCV(estimator=XGBRegressor(base_score=None, booster=None,
                                    callbacks=None, colsample_bylevel=None,
                                    colsample_bynode=None,
                                    colsample_bytree=None,
                                    early_stopping_rounds=None,
                                    enable_categorical=False, eval_metric=None,
                                    feature_types=None, gamma=None, gpu_id=None,
                                    grow_policy=None, importance_type=None,
                                    interaction_constraints=None,
                                    learning_rate=None, max_bin=None,
                                    max_cat_threshold=None,
                                    max_cat_to_onehot=None, max_delta_step=None,
                                    max_depth=None, max_leaves=None,
                                    min_child_weight=None, missing=nan,
                                    monotone_constraints=None,
                                    n_estimators=5000, n_jobs=None, nthread=-1,
                                    num_parallel_tree=None, predictor=None, ...),
             param_grid={'learning_rate': [0.01, 0.001, 0.005, 0.0001],
                         'max_depth': [2, 3, 4, 5]})

### Evaluation

Mean Absolute Percentage Error

In [72]:
model = grid.best_estimator_

In [74]:
def mean_absolute_percentage_error(y_true, y_pred):
    y_true, y_pred = np.array(y_true), np.array(y_pred)
    return np.mean(np.abs((y_true - y_pred) / y_true)) * 100

In [75]:
mean_absolute_percentage_error(y_test, model.predict(X_test))

62.617570109480226

### Saving Models

In [76]:
model.save_model('model.json')

In [77]:
from joblib import dump
dump(encoder, 'encoder.joblib')

['encoder.joblib']

# Analysis

From the evaluation with MAPE as the metric, we got a result of 62.62% of error. This result, while it's not really bad for starters in my opinion, it also means that our model is still unable to accurately forecast the demand for our items. The cause of this result might be because of several things that have been discussed in its own section (`Preprocess`, `Feature Engineering`, `Training`) and also because of the way we are defining the features for our model (that is, in my opinion, a bit far to be called as time-series feature).

A list of possible further improvements, either to the implementation or to the workflow, will be provided in the README of the repository.